In [5]:
from dotenv import load_dotenv

load_dotenv()

True

In [29]:
import random

In [7]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

scope = "user-library-read,playlist-modify-public"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [24]:
playlists = sp.current_user_playlists()['items']

In [25]:
playlist_uris = {playlist['name']: playlist['uri'] for playlist in playlists}
playlist_uris

{'test': 'spotify:playlist:6hgI2HKwU3bRjtWhYqBU7L',
 'Driving': 'spotify:playlist:33YvCoH4C75DHcCuqOkNM1',
 'Running': 'spotify:playlist:4Xq1xbmWHUsCT65RUkVesr',
 'Cotton Eyed Joe': 'spotify:playlist:1Jht9TdnfAN4I1z2yIweQr',
 'This Is Gaelic Storm': 'spotify:playlist:37i9dQZF1DZ06evO345fo4',
 "Reddit's Hype Playlist": 'spotify:playlist:7CseXkRUYWMS6jm0VafGlN'}

In [26]:
driving = playlist_uris['Driving']
test = playlist_uris['test']

In [54]:
number_of_songs = sp.playlist_items(driving, limit=1)['total']
number_of_songs

357

In [55]:
playlist_song_order = list(range(number_of_songs))

In [56]:
random.shuffle(playlist_song_order)
playlist_song_order

[98,
 1,
 234,
 89,
 200,
 305,
 170,
 11,
 35,
 46,
 124,
 198,
 145,
 167,
 244,
 77,
 97,
 282,
 39,
 208,
 336,
 203,
 94,
 191,
 180,
 226,
 160,
 156,
 297,
 112,
 228,
 175,
 49,
 335,
 2,
 0,
 65,
 52,
 138,
 181,
 230,
 57,
 243,
 134,
 309,
 22,
 186,
 5,
 223,
 276,
 275,
 304,
 146,
 257,
 201,
 279,
 339,
 169,
 233,
 351,
 121,
 227,
 73,
 64,
 263,
 274,
 266,
 293,
 245,
 356,
 139,
 144,
 303,
 140,
 256,
 204,
 174,
 299,
 150,
 206,
 178,
 222,
 21,
 116,
 59,
 273,
 72,
 19,
 44,
 37,
 12,
 123,
 108,
 267,
 281,
 168,
 152,
 280,
 79,
 43,
 87,
 315,
 10,
 26,
 7,
 177,
 284,
 105,
 196,
 4,
 76,
 15,
 290,
 47,
 81,
 218,
 193,
 337,
 14,
 62,
 215,
 185,
 286,
 189,
 172,
 126,
 67,
 125,
 28,
 340,
 48,
 294,
 155,
 349,
 74,
 241,
 50,
 285,
 55,
 213,
 164,
 352,
 41,
 308,
 348,
 130,
 136,
 330,
 122,
 25,
 133,
 347,
 85,
 197,
 120,
 159,
 106,
 344,
 102,
 260,
 3,
 300,
 142,
 176,
 60,
 327,
 91,
 100,
 68,
 42,
 182,
 252,
 183,
 137,
 34,
 71,
 127,
 

In [57]:
while len(playlist_song_order) > 0:
    song_to_move = playlist_song_order.pop()
    for i, value in enumerate(playlist_song_order):
        if value < song_to_move:
            playlist_song_order[i] += 1
    sp.playlist_reorder_items(driving, song_to_move, 0)
